In [182]:
from selenium import webdriver
import pandas as pd
import numpy as np

## Get Links:

In [241]:
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=chrome_options, executable_path="C:\chromedriver.exe")
driver.get('https://www.instagram.com/natgeo/?hl=en')

import time
from bs4 import BeautifulSoup
import re


links = []
scripts = []

while len(links) < 1000:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(2)
    
    page_html = driver.page_source
    data = BeautifulSoup(page_html, 'html.parser')
    
    body = data.find('body')
    script = body.find('span')
    
    for link in script.findAll('a'):
         if re.match("/p", link.get('href')):
            links.append(link.get('href').lstrip('/p/').rstrip('/'))
            
    links = list(dict.fromkeys(links))

In [242]:
#this includes videos
links

['B4k39YmnlKE',
 'B4kjoNqjdH0',
 'B4kQtm9D4kf',
 'B4j6iZ8g4TM',
 'B4jmA-igV8X',
 'B4jLactgdS_',
 'B4i6NzSD2QR',
 'B4io5gJD6A1',
 'B4iXk1JJUW7',
 'B4iG2RgDv-6',
 'B4h2ETMj2-0',
 'B4hju0ZHiHb',
 'B4hScDFHrui',
 'B4hBRbhgmZB',
 'B4gmuVCoPRd',
 'B4gVX_doh2W',
 'B4gEI0-lUyF',
 'B4fy0t2liGX',
 'B4fhn7YDL4v',
 'B4fR4Qnje9k',
 'B4e-9qHAwXr',
 'B4ecdCCAmKs',
 'B4eJgGXj34f',
 'B4d4OlRIrBO',
 'B4dm-dJIstG',
 'B4dVq1jHn3D',
 'B4dEYqNJaci',
 'B4czDDkDkgC',
 'B4cglbrD7sy',
 'B4cMFAEDV1f',
 'B4b3kbfAvHl',
 'B4biRkHDhwC',
 'B4bNyFjjUx3',
 'B4a5Q8zDdx8',
 'B4ak0cvBKEO',
 'B4aRYURjOYZ',
 'B4Z70NxAmOX',
 'B4ZnWytAgFO',
 'B4ZSwfFAF25',
 'B4Y2jnxDOtH',
 'B4YiFFPD5wH',
 'B4YNjOrA1ae',
 'B4X5IoNHP6q',
 'B4XknyjACN2',
 'B4XQHcjgN_H',
 'B4W7mQsgzqP',
 'B4WnM4_geap',
 'B4WMpQUgizs',
 'B4V7W5ND2os',
 'B4VqBZhASk3',
 'B4VYxYnlRHj',
 'B4VHfyhAPgv',
 'B4U2NdxABg-',
 'B4Uk5TSAM7k',
 'B4UTmUXgXfw',
 'B4UCUp6jHaG',
 'B4Ts6U5j5wJ',
 'B4TYcguAZrZ',
 'B4TD9KigbBr',
 'B4SvnP_DuP4',
 'B4Sdp6ADSxv',
 'B4SGf5ngVVt',
 'B4RyDt

In [277]:
len(links)

1008

## Get Captions, Comments, and Likes:

In [325]:
import instaloader
L = instaloader.Instaloader()

posts = dict()

for count, post in enumerate(links):
    Post = instaloader.Post.from_shortcode(L.context, post)
    
    if not Post.is_video:
        posts[count] = [Post._asdict()['display_url'], Post.caption, Post.comments, Post.likes]

In [326]:
post_info = pd.DataFrame.from_dict(posts, orient = 'index', columns = ['display_url', 'caption', 'comments', 'likes'])
post_info

,display_url,caption,comments,likes
0,https://instagram.fftw1-1.fna.fbcdn.net/vp/801...,Photo by Erika Larsen @erikalarsen888 | Roxane...,360,47452
1,https://instagram.fftw1-1.fna.fbcdn.net/vp/8c4...,Photo by David Guttenfelder @dguttenfelder // ...,336,85229
2,https://instagram.fftw1-1.fna.fbcdn.net/vp/764...,Photo by George Steinmetz @geosteinmetz | A ca...,758,292916
4,https://instagram.fftw1-1.fna.fbcdn.net/vp/527...,Photo by Maggie Steber @maggiesteber | Veteran...,254,98153
5,https://instagram.fftw1-1.fna.fbcdn.net/vp/a33...,Photo by Anastasia Taylor-Lind @anastasiatl | ...,646,132572
...,...,...,...,...
1003,https://instagram.fftw1-1.fna.fbcdn.net/vp/966...,Photo by Brian Skerry @brianskerry | A threshe...,5255,531996
1004,https://instagram.fftw1-1.fna.fbcdn.net/vp/24b...,Photo by Stephen Wilkes @stephenwilkes | It is...,3792,826978
1005,https://instagram.fftw1-1.fna.fbcdn.net/vp/8a1...,Photo by Michael Yamashita @yamashitaphoto | O...,2070,754997
1006,https://instagram.fftw1-1.fna.fbcdn.net/vp/96f...,"Photo by Joel Sartore @joelsartore | Hurry, be...",1292,448321


## Get Labels

In [327]:
from google.cloud import vision
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:\My First Project-fe9e86c51751.json"
client = vision.ImageAnnotatorClient()

In [328]:
def get_labels(url):
    response = client.annotate_image({
      'image': {'source': {'image_uri': url}},
      'features': [{'type': "LABEL_DETECTION"}],
    })
    
    labels = [annotation.description for annotation in response.label_annotations]
    
    return labels

In [329]:
post_info['labels'] = post_info['display_url'].map(get_labels)

In [335]:
post_info = post_info.reset_index().drop('index', axis = 1)

In [336]:
post_info

,display_url,caption,comments,likes,labels
0,https://instagram.fftw1-1.fna.fbcdn.net/vp/801...,Photo by Erika Larsen @erikalarsen888 | Roxane...,360,47452,"[Clothing, Dress shirt, Sleeve, Shirt, Formal ..."
1,https://instagram.fftw1-1.fna.fbcdn.net/vp/8c4...,Photo by David Guttenfelder @dguttenfelder // ...,336,85229,"[Bicycle, Pink, Bicycle wheel, Vehicle, Cyclin..."
2,https://instagram.fftw1-1.fna.fbcdn.net/vp/764...,Photo by George Steinmetz @geosteinmetz | A ca...,758,292916,"[Yellow, Sky, Sand, Dune, Landscape, Heat, Aeo..."
3,https://instagram.fftw1-1.fna.fbcdn.net/vp/527...,Photo by Maggie Steber @maggiesteber | Veteran...,254,98153,"[Design, Room, Photography, Material property,..."
4,https://instagram.fftw1-1.fna.fbcdn.net/vp/a33...,Photo by Anastasia Taylor-Lind @anastasiatl | ...,646,132572,"[Room, Photography, Sitting]"
...,...,...,...,...,...
942,https://instagram.fftw1-1.fna.fbcdn.net/vp/966...,Photo by Brian Skerry @brianskerry | A threshe...,5255,531996,"[Fish, Shark, Blue, Water, Great white shark, ..."
943,https://instagram.fftw1-1.fna.fbcdn.net/vp/24b...,Photo by Stephen Wilkes @stephenwilkes | It is...,3792,826978,"[Elephant, Elephants and Mammoths, Mammal, Ter..."
944,https://instagram.fftw1-1.fna.fbcdn.net/vp/8a1...,Photo by Michael Yamashita @yamashitaphoto | O...,2070,754997,"[Nature, Natural landscape, Reflection, Wilder..."
945,https://instagram.fftw1-1.fna.fbcdn.net/vp/96f...,"Photo by Joel Sartore @joelsartore | Hurry, be...",1292,448321,"[Sky, Blue, Tree, Nature, Night, Branch, Woody..."


In [337]:
import pickle

pickle.dump( post_info, open( "save.p", "wb" ) )